# Basketball-Reference Scraper Overview
The following code will walk you through how to scrape NBA player and game data from www.basketball-reference.com and input it into a Pandas database. In other scripts within this directory, we will be using the data captured here to run analyses that will help the average user ask both broad and specific questions related to the NBA. We will probe on what statistics and criteria are important for an NBA team to win an NBA championship, how the league has evolved year-over-year, touch on the GOAT debate, and ultimately, build algorithms that can (hopefully) help us all beat Vegas lines consistently so that we can all retire from our day jobs and gamble on the NBA for the rest of our careers. 

None of this could have been done without the tireless and comprehensive effort of those who work at [Basketball Reference](http://www.basketball-reference.com) providing an open-source, API-friendly database containing millions of datapoints from which the entirety of this codebase is built. 

For any questions/concerns, feel free to reach out to me directly at rahim.hashim@columbia.edu. And in the case that this is useful to anyone for future projects, please give credit where credit is due, both to [Basketball Reference](http://www.basketball-reference.com) and myself. Enjoy!

***
## The Basics
__Jupyter Notebook__: All of the following code is hosted in a Python 3 Jupyter Notebook. It is recommended to use Anaconda to access the Notebook in order to have synchronously have access to all Python Libraries used in the rest of the code. 

In order to execute and compile code in the notebook, go to the desired code box and press _Shift_ + _Enter_ at the same time. All code below is recommended to be executed from top to bottom in order.

__Python Libraries__: Python is a beautiful language for a number of reasons, one of which is it's immense
amount of pre-built libraries that do much of the heavy lifting in any web-scraping /
data analysis project. When getting familiar with Python and starting a new project, be
sure to look through the internet for a Python library that may help. A comprehensive list
that I often refer to before starting a project is here: [https://github.com/vinta/awesome-python](https://github.com/vinta/awesome-python)

__Installing Libraries__: In case you receive an error upon trying to execute the following box, such as _ModuleNotFoundError: No module named 'numpy'_, go back to your terminal and open a new tab, and install the library using pip: _pip install numpy_

In [1]:
%reload_ext autoreload
import re
import os
import sys
import numpy as np
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/'

# add (non-Python) helper functions
def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/

  helper_dir_path = ROOT + 'helper/'
  print('\nHelpers:')
  pprint(sorted(os.listdir(helper_dir_path)))
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

Mounted at /content/drive
/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms

Helpers:
['Regions.py',
 'TeamNames.py',
 '__pycache__',
 'bettingLinesScraper.py',
 'gameLogScraper.py',
 'gamelogscraper.ipynb',
 'meta_info_scraper.py',
 'player_info_scraper.py',
 'player_scraper.py',
 'player_table_scraper.py',
 'seasonScraper.py',
 'teamsScraper.py']


***
## Creating Databases
Pandas databases are a powerful tool to query large amounts of data, as we will be doing here. For that reason, we are going to insert all of the data scraped above into a Pandas database. The below code will take player overview data from playerHash and insert it into player_df<br>
>For documentation on pandas: https://pypi.org/project/pandas/

***
## Scraping Player Data
### Biometrics and season + career statistics

playerScraper and metaDataScraper will be doing most of the work to scrape data on each player's background and physical attributes.<br>
> Example Overview Source (last name starting with a): https://www.basketball-reference.com/players/a/<br>
> Example meta-data (Karim Abdul-Jabbar): https://www.basketball-reference.com/players/a/abdulka01.html<br>
> For documentation on requests(): https://realpython.com/python-requests/<br>
> For documentation on BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

___Time Estimates:___ This is the most computationally-intensive function in the program, requiring many url requests in order to complete.<br>
>*Without threading:* ~1hr<br>
>*With threading:* ~15min<br>

In [2]:
from player_scraper import scrape_all_players

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/'
df_players_meta, df_players_data = scrape_all_players(ROOT, THREAD_FLAG=True)

Running meta_info_scraper.py
  Start Time: 18:30:52.35
  Threading activated...
	  x' Players Captured:  0
	  q' Players Captured:  6
	  u' Players Captured:  11
	  z' Players Captured:  20
	  y' Players Captured:  19
	  i' Players Captured:  26
	  v' Players Captured:  59
	  o' Players Captured:  95
	  n' Players Captured:  105
	  e' Players Captured:  106
	  f' Players Captured:  148
	  k' Players Captured:  170
	  t' Players Captured:  193
	  l' Players Captured:  195
	  a' Players Captured:  172
	  p' Players Captured:  217
	  d' Players Captured:  242
	  r' Players Captured:  253
	  g' Players Captured:  246
	  j' Players Captured:  238
	  c' Players Captured:  301
	  h' Players Captured:  351
	  w' Players Captured:  373
	  s' Players Captured:  421
	  b' Players Captured:  468


  0%|          | 8/4898 [00:00<01:02, 78.29it/s]

	  m' Players Captured:  463
  Run Time: 115.18 min
  Concatenating DataFrames


100%|██████████| 4898/4898 [2:46:43<00:00,  2.04s/it]


  Concatenating complete
Saving players_df_meta.pkl
  Path: /content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/data/players_df_meta.pkl
Saving players_df_data.pkl
  Path: /content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/data/players_df_data.pkl
  Size (meta info): 4.6MB
  Size (player data): 1.3GB
Complete.


***
## Example Queries (Simple)

The following are example queries we can make across all of the generated tables. As can be seen below, the structure of the DataFrame allows for immense flexibility and speed gains as compared to looking at the website itself. We will utilize this structure for more specific trend-, team-, and era- related investigations.

In [3]:
df_players_meta.head(0)

,player_name,draft_year,retire_year,height,weight,birth_date,college,birthCountry,birthCity,birthState,shootingHand,highSchool,highSchoolCity,highSchoolState,highSchoolCountry,draftTeam,draftRound,draftRoundPick,draftOverallPick


In [4]:
# Player Meta Query
df_large = df_players_meta.loc[(df_players_meta['height']>80) & 
                   (df_players_meta['weight']>30)]

df_large = df_large.dropna(how='all', axis='columns') # drops all columns that are empty
display(df_large[df_large['weight'] == df_large['weight'].max()])

,player_name,draft_year,retire_year,height,weight,birth_date,college,birthCountry,birthCity,birthState,shootingHand,highSchool,highSchoolCity,highSchoolState,highSchoolCountry,draftTeam,draftRound,draftRoundPick,draftOverallPick
0,Sim Bhullar,2015,2015,89,360,"December 2, 1992",New Mexico State,Canada,Ontario,NaN,Right,Huntington Prep,Huntington,West Virginia,United States of America,NaN,NaN,NaN,NaN


In [5]:
df_players_data.head(5)

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,ft_per_g,fta_per_g,ft_pct,trb_per_g,ast_per_g,pf_per_g,pts_per_g,mp,fg,fga,ft,fta,trb,ast,pf,pts,per,ts_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,DUMMY,ows,dws,...,blk_per_mp,tov_per_mp,fg_per_poss,fga_per_poss,fg3_per_poss,fg3a_per_poss,fg2_per_poss,fg2a_per_poss,ft_per_poss,fta_per_poss,orb_per_poss,drb_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,,off_rtg,def_rtg,info,header_pos_estimates,header_plus_minus,header_turnovers,header_fouls,header_drawn,header_misc2,zzz,pct_fga,fg_pct_dist,pct_ast,dunks,corner3,heaves,college_id,team_name,salary,trp_dbl
0,game,season,Joe Fabel,1946-47,29.0,PIT,BAA,NaN,30.0,NaN,NaN,0.8,3.2,.260,0.4,0.9,.500,NaN,0.1,2.1,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,season,Joe Fabel,Career,NaN,NaN,BAA,NaN,30.0,NaN,NaN,0.8,3.2,.260,0.4,0.9,.500,NaN,0.1,2.1,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,season,Joe Fabel,1946-47,29.0,PIT,BAA,NaN,30.0,NaN,NaN,NaN,NaN,.260,NaN,NaN,.500,NaN,NaN,NaN,NaN,NaN,25.0,96.0,13.0,26.0,NaN,2.0,64.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,season,Joe Fabel,Career,NaN,NaN,BAA,NaN,30.0,NaN,NaN,NaN,NaN,.260,NaN,NaN,.500,NaN,NaN,NaN,NaN,NaN,25.0,96.0,13.0,26.0,NaN,2.0,64.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,season,Joe Fabel,1946-47,29.0,PIT,BAA,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.293,.271,NaN,NaN,NaN,NaN,NaN,-0.1,-0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Player Data Query
'''
df_career = df_players_data.loc[(df_players_data['season']=='Career') & 
                   (df_players_data['season_playoffs']=='season') &
                   (df_players_data['data_type']=='advanced')]
'''
df_players_data['mp_clean'] = df_players_data['mp_per_g'].apply(float)
#df_career[df_career['ws_per_48'].notna()].sort_values('ws_per_48', ascending=False)
df_players_data[(df_players_data['season'] == '2020-21') & 
                (df_players_data['data_type'] == 'per_game') &
                (df_players_data['mp_clean'] > 40)]

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,ft_per_g,fta_per_g,ft_pct,trb_per_g,ast_per_g,pf_per_g,pts_per_g,mp,fg,fga,ft,fta,trb,ast,pf,pts,per,ts_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,ows,dws,ws,...,pct_3,pct_4,pct_5,plus_minus_on,plus_minus_net,tov_bad_pass,tov_lost_ball,fouls_shooting,fouls_offensive,drawn_shooting,drawn_offensive,astd_pts,and1s,own_shots_blk,avg_dist,pct_fga_fg2a,pct_fga_00_03,pct_fga_03_10,pct_fga_10_16,pct_fga_16_xx,pct_fga_fg3a,fg_pct_fg2a,fg_pct_00_03,fg_pct_03_10,fg_pct_10_16,fg_pct_16_xx,fg_pct_fg3a,pct_ast_fg2,pct_ast_fg3,pct_fga_dunk,fg_dunk,pct_fg3a_corner3,fg_pct_corner3,fg3a_heave,fg3_heave,college_id,team_name,salary,trp_dbl,mp_clean
21,per_game,playoffs,Paul George,2020-21,30.0,LAC,NBA,SF,19.0,19.0,40.8,8.7,19.7,.441,6.8,8.1,.844,9.6,5.4,3.1,26.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.8
17,per_game,playoffs,Damian Lillard,2020-21,30.0,POR,NBA,PG,6.0,6.0,41.3,10.3,22.3,.463,7.8,8.3,.940,4.3,10.2,1.5,34.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.3
5,per_game,playoffs,Luka Dončić,2020-21,21.0,DAL,NBA,PG,7.0,7.0,40.1,13.7,28.0,.490,3.9,7.3,.529,7.9,10.3,2.4,35.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.1
23,per_game,playoffs,Kevin Durant,2020-21,32.0,BRK,NBA,PF,12.0,12.0,40.4,12.1,23.5,.514,7.3,8.4,.871,9.3,4.4,2.6,34.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.4
7,per_game,playoffs,Devin Booker,2020-21,24.0,PHO,NBA,SG,22.0,22.0,40.4,9.6,21.5,.447,6.0,6.7,.905,5.6,4.5,3.5,27.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.4
15,per_game,playoffs,Khris Middleton,2020-21,29.0,MIL,NBA,SF,23.0,23.0,40.1,8.6,19.7,.438,3.7,4.2,.887,7.6,5.1,2.5,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.1
3,per_game,playoffs,Ja Morant,2020-21,21.0,MEM,NBA,PG,5.0,5.0,40.6,11.0,22.6,.487,6.2,8.0,.775,4.8,8.2,1.8,30.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.6


***
## Single Player Queries

If you don't need to scrape the entire basketball-reference website but instead just want information for one player, you can use the following command.

In [39]:
from player_info_scraper import player_info_scraper
from player_scraper import single_player_scraper

pd.set_option('display.max_rows', None)
single_player_info, single_player_df = single_player_scraper()
single_player_df

Player Name: Michael Jordan
Searching for name in: https://www.basketball-reference.com/players/j
Michael Jordan found. DataFrames generated


,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,fg2_per_g,fg2a_per_g,fg2_pct,efg_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,mp,fg,fga,fg3,fg3a,fg2,fg2a,...,pct_1,pct_2,pct_3,pct_4,pct_5,plus_minus_on,plus_minus_net,tov_bad_pass,tov_lost_ball,fouls_shooting,fouls_offensive,drawn_shooting,drawn_offensive,astd_pts,and1s,own_shots_blk,avg_dist,pct_fga_fg2a,pct_fga_00_03,pct_fga_03_10,pct_fga_10_16,pct_fga_16_xx,pct_fga_fg3a,fg_pct_fg2a,fg_pct_00_03,fg_pct_03_10,fg_pct_10_16,fg_pct_16_xx,fg_pct_fg3a,pct_ast_fg2,pct_ast_fg3,pct_fga_dunk,fg_dunk,pct_fg3a_corner3,fg_pct_corner3,fg3a_heave,fg3_heave,college_id,team_name,salary
0,per_game,season,Michael Jordan,1984-85,21,CHI,NBA,SG,82,82,38.3,10.2,19.8,.515,0.1,0.6,.173,10.1,19.2,.526,.518,7.7,9.1,.845,2.0,4.5,6.5,5.9,2.4,0.8,3.5,3.5,28.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,per_game,season,Michael Jordan,1985-86,22,CHI,NBA,SG,18,7,25.1,8.3,18.2,.457,0.2,1.0,.167,8.2,17.2,.474,.462,5.8,6.9,.840,1.3,2.3,3.6,2.9,2.1,1.2,2.5,2.6,22.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,per_game,season,Michael Jordan,1986-87,23,CHI,NBA,SG,82,82,40.0,13.4,27.8,.482,0.1,0.8,.182,13.2,27.0,.491,.484,10.2,11.9,.857,2.0,3.2,5.2,4.6,2.9,1.5,3.3,2.9,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,per_game,season,Michael Jordan,1987-88,24,CHI,NBA,SG,82,82,40.4,13.0,24.4,.535,0.1,0.6,.132,13.0,23.7,.546,.537,8.8,10.5,.841,1.7,3.8,5.5,5.9,3.2,1.6,3.1,3.3,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,per_game,season,Michael Jordan,1988-89,25,CHI,NBA,SG,81,81,40.2,11.9,22.2,.538,0.3,1.2,.276,11.6,21.0,.553,.546,8.3,9.8,.850,1.8,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,per_game,season,Michael Jordan,1989-90,26,CHI,NBA,SG,82,82,39.0,12.6,24.0,.526,1.1,3.0,.376,11.5,21.0,.548,.550,7.2,8.5,.848,1.7,5.1,6.9,6.3,2.8,0.7,3.0,2.9,33.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,per_game,season,Michael Jordan,1990-91,27,CHI,NBA,SG,82,82,37.0,12.1,22.4,.539,0.4,1.1,.312,11.7,21.3,.551,.547,7.0,8.2,.851,1.4,4.6,6.0,5.5,2.7,1.0,2.5,2.8,31.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,per_game,season,Michael Jordan,1991-92,28,CHI,NBA,SG,80,80,38.8,11.8,22.7,.519,0.3,1.3,.270,11.5,21.5,.533,.526,6.1,7.4,.832,1.1,5.3,6.4,6.1,2.3,0.9,2.5,2.5,30.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,per_game,season,Michael Jordan,1992-93,29,CHI,NBA,SG,78,78,39.3,12.7,25.7,.495,1.0,2.9,.352,11.7,22.7,.514,.515,6.1,7.3,.837,1.7,5.0,6.7,5.5,2.8,0.8,2.7,2.4,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,per

***
## Scraping Game Data
### Game-logs and team statistics

In [ ]:
# WIP

***
## Data Organization
To help us understand how all the data is organized, here's a visual:

In [ ]:
df_career = df_players_data.loc[(df_players_data['season']=='Career') & 
                   (df_players_data['season_playoffs']=='season') &
                   (df_players_data['data_type']=='advanced')]

df_career.dropna(axis='columns')

***
## Meta-Data Analysis
Now that we've scraped all the meta-info on each player, we can start running analyses.

Below, a few simple analyses are included to help you get started. The first set of graphs examine height distribution (left), weight distribution (middle), and shooting handedness (right).

In [ ]:
from Regions import stateDict #stateDict is a Dictionary to help with geography-based analyses
def metaPlot():
  
  height_list = df_players_meta['height'].tolist()
  weight_list = df_players_meta['weight'].tolist()
  #rightCount = 0; leftCount = 0; noHandCount = 0

  #Plot Height Distribution (1, Left)
  f, ax = plt.subplots(1,2)
  #Sets default plot size
  plt.rcParams['figure.figsize'] = (10,8)
  n1, bins1, patches1 = ax[0].hist(height_list, bins=20, density=True, histtype='bar', ec='black')
  #Converting y-axis labels from decimals to percents
  y_vals = ax[0].get_yticks(); ax[0].set_yticklabels(['{:3.1f}%'.format(y*100) for y in y_vals])
  #Converting x-axis labels from inches back to feet
  xticks1 = ['5-0', '5-6', '6-0', '6-6', '7-0', '7-6', '8-0']
  ax[0].set_xticks([60, 66, 72, 78, 84, 90, 96])
  ax[0].set_xticklabels(xticks1)
  ax[0].set_xlim([56,100])
  ax[0].set_xlabel('Height', fontweight='bold', labelpad=10)
  ax[0].set_ylabel('Percent of Players', fontweight='bold', labelpad=10)

  #Plot Weight Distribution (1, Middle)
  ax[1].hist(weight_list, bins='auto', density=True, histtype='bar', ec='black')
  y_vals = ax[1].get_yticks()
  ax[1].set_yticklabels(['{:3.1f}%'.format(y*100) for y in y_vals])
  xticks2 = ['150', '180', '210', '240', '270', '300', '330']
  ax[1].set_xticks([150, 180, 210, 240, 270, 300, 330])
  ax[1].set_xticklabels(xticks2)
  ax[1].set_xlim([120,360])
  ax[1].set_xlabel('Weight', fontweight='bold', labelpad=10)
  ax[1].set_ylabel('Percent of Players', fontweight='bold', labelpad=10)
  
  plt.tight_layout(pad=0.05, w_pad=4, h_pad=1.0)
  f.set_size_inches(18.5, 10.5, forward=True)
  plt.show()
        
metaPlot()

In [ ]:
def geographyPlot():
  countryList = df_players_meta['birthCountry'].tolist()
  #countryList contains all players born in ex-US
  countryList = filter(lambda x: x != 'United States of America', countryList)
  countryList = filter(lambda x: x != '', countryList)
  countryHash = dict(Counter(countryList))
  countryHash = OrderedDict(sorted(countryHash.items(), reverse=True, key=lambda t: t[1]))

  pprint(countryHash.keys())

  #Plot Birth Countries of non-US-Born Players (3)
  f, ax = plt.subplots(1)
  countryList = countryHash.keys(); countryVals = countryHash.values()
  ax.bar(np.arange(len(countryList)), countryVals, ec='black')
  ax.set_xticks(np.arange(len(countryList)))
  ax.set_xticklabels(countryList, rotation=90, ha='right', fontsize=7)
  ax.set_xlabel('Country of Birth', fontweight='bold', labelpad=10)
  ax.set_ylabel('Number of Players', fontweight='bold', labelpad=10)
  
  f.set_size_inches(18.5, 10.5, forward=True)
  plt.show()
    
geographyPlot()